## 実践タスク: ベスト画像を地図に表示
### 課題: 前回抽出したLandsat 9の最も雲量の少ない画像を、標準的な衛星画像の上に、カラー合成（自然色）で重ねて表示しなさい。
---
GEE Python APIで地図表示を行うには、**folium**や**ipyleaflet**といったライブラリと、Earth Engineが提供する**ee.Image.getMapId()**メソッドを組み合わせて使用するのが一般的です。

ここでは、セットアップが簡単な**folium**を使った表示方法で実践します。


1. 必要なライブラリのインストール

In [ ]:
import ee
import datetime
import folium
from folium import plugins

In [ ]:
ee.Initialize(project='earth-change-analysis')

## 1. 地域の定義

In [ ]:
biwako_point = ee.Geometry.Point([136.17, 35.10])
longitude, latitude = biwako_point.coordinates().getInfo() # 地図の中心座標を取得

## 2. 検索期間の定義 (過去1年間)

In [ ]:
end_date = '2025-11-17' 
start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

## 3. 画像コレクションのロード

In [ ]:
l9_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

## 4. フィルタリングと並べ替え (前回と同じコード)

In [ ]:
filtered_collection = l9_collection \
    .filterDate(start_date, end_date) \
    .filterBounds(biwako_point)
sorted_collection = filtered_collection.sort('CLOUD_COVER', True)
best_image = sorted_collection.first()

## 5. 結果の確認 (再掲)

In [ ]:
if best_image is None:
    print("条件に一致する画像が見つかりませんでした。")
else:
    cloud_cover = best_image.get('CLOUD_COVER').getInfo()
    print(f"雲量 {cloud_cover:.2f}% のベスト画像を表示します。")

# --- ここから地図表示のステップ ---

### 5-1. 表示パラメータの定義 (Landsat C2 Level 2 の自然色)

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Landsat C2 L2のバンド名はB4(赤), B3(緑), B2(青)
vis_params = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'], # バンドの指定
        'min': 0,
        'max': 12408.557332596974, # 反射率のクリップ値 (コントラスト調整)
        'gamma': 1.3
}

### 5-2. Foliumマップの作成

In [ ]:
my_map = folium.Map(
        location=[latitude, longitude],   # 地図の中心座標
        zoom_start=10,                    # 初期ズームレベル (琵琶湖周辺)
        tiles=basemaps['Google Satellite Hybrid'] # 初期タイルを設定
    )

### 5-3. ベースマップの追加 (初期タイル以外もレイヤーコントロールに追加)

In [ ]:
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite'].add_to(my_map)
# basemaps['Google Satellite Hybrid'] は既に初期タイルとして設定されている
basemaps['Esri Satellite'].add_to(my_map)

### 5-4. GEE 画像レイヤーの追加

In [ ]:
map_id_dict = best_image.getMapId(vis_params)
tile_url = map_id_dict['tile_fetcher'].url_format

In [ ]:
folium.TileLayer(
    tiles=tile_url,
    attr='Map data © Google Earth Engine',
    name='Best Landsat Image', # レイヤー名を修正
    overlay=True,
    control=True
).add_to(my_map)

### 5-5. レイヤーコントロールの追加

In [ ]:
folium.LayerControl().add_to(my_map) # 変数を m から my_map に修正

### 5-6. マップの表示

In [ ]:
display(my_map)

#### 画像の真の明るさを確認する
>[!note} これまで、私たちは Landsat の標準的な知識に基づいて max の値を推測してきました。しかし、最適なコントラストを得るには、画像データが実際に持っているピクセル値の範囲を知る必要があります。
>これを実現するために、GEEの統計機能を使って、抽出した best_image のピクセル値の範囲を調べましょう。特に、画像のほとんどの部分が収まる98パーセンタイルの値を取得します。これにより、ノイズや極端に明るい雲の端などを除外し、適切な max 値がわかります。
>以下のコードを実行し、推奨される新しい max 値（ピクセル値）を取得してください。



In [ ]:
# GEEの統計機能を使って、表示バンドの98パーセンタイルを計算する
stats = best_image.select('SR_B4', 'SR_B3', 'SR_B2').reduceRegion(
    reducer=ee.Reducer.percentile([98]), # 98パーセンタイルを使用
    geometry=best_image.geometry(), # 画像全体の範囲で計算
    scale=30, # Landsatの解像度
    maxPixels=1e9 # 計算に使用する最大ピクセル数
)

# SR_B4バンドの98パーセンタイル値を取得
# (自然色バンドの値は近いため、一つ取得すれば十分です)
max_value_98 = stats.get('SR_B4').getInfo()

print(f"統計的に推奨される新しい max 値: {max_value_98}")

---
title: "GEEにおけるピクセル値とコントラスト調整"
description: "Landsat画像のピクセル値の意味、min/maxの設定とクリッピングの関係、98パーセンタイルによる最適化手法を解説。"
tags: ["GEE", "ピクセル値", "リモートセンシング", "min/max", "可視化", "Landsat", "Obsidian"]
date: 2025-11-19
author: "Seiichi"
category: "GEE基礎解説"
---

## ピクセル値とは何か？

GEEで扱うリモートセンシング画像（例：Landsat）では、各ピクセルは「特定の波長の光をどれだけ反射したか」を数値で表しています。

| ピクセル値の範囲 | 反射率の意味 | 表示上の見え方 | 主な地表例 |
|------------------|--------------|----------------|-------------|
| 0〜1000          | 非常に暗い   | 黒〜濃い青     | 水域、影、焼け跡 |
| 1000〜3000       | やや暗い     | 暗い緑〜灰色   | 森林、湿地、アスファルト |
| 3000〜6000       | 中程度       | 緑〜黄          | 草地、農地、裸地 |
| 6000〜9000       | 明るい       | 黄〜白          | 砂地、都市、雲 |
| 9000〜10000      |非常に明るい  | 白              | 雲の端、雪氷、センサ飽和 |

---

## なぜ max 値が重要なのか？

- `vis_params` の `max` 値が低すぎると、明るい地表（例：砂浜、コンクリート、雲）がすべて白く表示されてしまい、**クリッピング（飽和）**が発生します。
- 逆に高すぎると、全体が暗くなり、**コントラストが失われます**。

---

## 最適な max 値を求める方法（98パーセンタイル）

```python
# 可視化対象のバンドを選択（例：自然色）
bands = ['SR_B4', 'SR_B3', 'SR_B2']

# 98パーセンタイルを計算
stats = best_image.select(bands).reduceRegion(
    reducer=ee.Reducer.percentile([98]),
    geometry=best_image.geometry(),
    scale=30,
    maxPixels=1e9
)

# SR_B4（赤バンド）の98パーセンタイル値を取得
max_value_98 = stats.get('SR_B4').getInfo()
print(f"推奨される max 値: {max_value_98}")
```